SPDX-License-Identifier: Apache-2.0

This project was developed by Linda Mendez, a student at the Universidad Nacional de Colombia. 


This notebook introduces a musical box, explains the implementation with configurability, and generates a tapeout-ready(GDS) design with open-source tools and PDK.

The musical box was a project developed with peripherals such as a disk, keys that represent an octave in the piano, buttons to control the volume, and the option to play freely or record the melodies played on the keys to be played when the user wants. The complete project was implemented in VHDL, but only the keyboard was implemented in Verilog and used in this notebook.


# Introduction

This project was designed for use by blind people, so the musical notes are written in Braille on the keyboard. The motivation is to provide a tool for blind people to play their melodies and listen to them. 
This program has as input an array of keys, where each time a key is pressed, its bit changes to one, the simulation also receives the time that this key was pressed since this is a physical input variable. In the physical implementation, the musical notes were written in braille.

![bconfig](https://github.com/LindaM123/caja_musica/blob/main/Images/pianito.png?raw=true)





# How does it work?

The music box receives the key that is pressed, it also has a reset button. When a key is pressed, the output of the box is a signal that has the frequency of the chosen note, to produce signals with different frequencies frequency dividers are used.


![bconfig](https://github.com/LindaM123/caja_musica/blob/main/Images/state_machine.png?raw=true)



In [1]:
!rm -rf caja_musica
!git clone https://github.com/LindaM123/caja_musica.git
!rm -rf musica
!mv caja_musica/musica .
!rm -rf caja_musica

Cloning into 'caja_musica'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 88 (delta 36), reused 69 (delta 23), pack-reused 0
Unpacking objects: 100% (88/88), 6.22 MiB | 10.59 MiB/s, done.


In [2]:
#@title Install dependencies {display-mode: "form"}
#@markdown - Click the ▷ button to setup the digital design environment based on [conda-eda](https://github.com/hdl/conda-eda).

openlane_version = 'latest' #@param {type:"string"}
open_pdks_version = 'latest' #@param {type:"string"}

if openlane_version == 'latest':
  openlane_version = ''
if open_pdks_version == 'latest':
  open_pdks_version = ''



import os
import pathlib

!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!CI=0 bin/micromamba install --yes --prefix $CONDA_PREFIX \
                     --channel litex-hub \
                     --channel main \
                     openlane={openlane_version} \
                     open_pdks.sky130a={open_pdks_version} 
!python -m pip install gdstk gdstk
PATH = os.environ['PATH']
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}


                                           __
          __  ______ ___  ____ _____ ___  / /_  ____ _
         / / / / __ `__ \/ __ `/ __ `__ \/ __ \/ __ `/
        / /_/ / / / / / / /_/ / / / / / / /_/ / /_/ /
       / .___/_/ /_/ /_/\__,_/_/ /_/ /_/_.___/\__,_/
      /_/

Empty environment created at prefix: /content/conda-env

                                           __
          __  ______ ___  ____ _____ ___  / /_  ____ _
         / / / / __ `__ \/ __ `/ __ `__ \/ __ \/ __ `/
        / /_/ / / / / / / /_/ / / / / / / /_/ / /_/ /
       / .___/_/ /_/ /_/\__,_/_/ /_/ /_/_.___/\__,_/
      /_/


Pinned packages:
  - python 3.7*


Transaction

  Prefix: /content/conda-env

  Updating specs:

   - openlane=*
   - open_pdks.sky130a=*


  Package                              Version  Build                 Channel                 Size
────────────────────────────────────────────────────────────────────────────────────────────────────
  Install:
──────────────────────────────────────────

In [ ]:
%%writefile config.json
{
    "DESIGN_NAME": "principal",
    "VERILOG_FILES": "dir::musica/rtl/principal.v",
    "CLOCK_PERIOD": 20,
    "CLOCK_NET": "i_clk",
    "CLOCK_PORT": "i_clk",

    "FP_SIZING": "absolute",
    "DIE_AREA": "0 0 300 300"
}

Overwriting config.json


In [ ]:
%env PDK=sky130A
!flow.tcl -design .

env: PDK=sky130A
OpenLane 2023.03.01_0_ge10820ec-conda
All rights reserved. (c) 2020-2022 Efabless Corporation and contributors.
Available under the Apache License, version 2.0. See the LICENSE file for more details.

[INFO]: Using configuration in 'config.json'...
[INFO]: PDK Root: /content/conda-env/share/pdk
[INFO]: Process Design Kit: sky130A
[INFO]: Standard Cell Library: sky130_fd_sc_hd
[INFO]: Optimization Standard Cell Library: sky130_fd_sc_hd
[INFO]: Run Directory: /content/runs/RUN_2023.04.03_04.09.15
[INFO]: Preparing LEF files for the nom corner...
[INFO]: Preparing LEF files for the min corner...
[INFO]: Preparing LEF files for the max corner...
[STEP 1]
[INFO]: Running Synthesis (log: runs/RUN_2023.04.03_04.09.15/logs/synthesis/1-synthesis.log)...
[STEP 2]
[INFO]: Running Single-Corner Static Timing Analysis (log: runs/RUN_2023.04.03_04.09.15/logs/synthesis/2-sta.log)...
[STEP 3]
[INFO]: Running Initial Floorplanning (log: runs/RUN_2023.04.03_04.09.15/logs/floorplan/3-ini

In [ ]:
import pathlib
import gdstk

gdss = sorted(pathlib.Path('runs').glob('*/results/final/gds/*.gds'))
library = gdstk.read_gds(gdss[-1])
top_cells = library.top_level()
top_cells[0].write_svg('principal.svg')

In [ ]:
!pip install svgutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from IPython.display import SVG,display

display(SVG(filename='principal.svg'))

In [ ]:
import pandas as pd
import pathlib

pd.options.display.max_rows = None
final_summary_reports = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(final_summary_reports[-1])
df.transpose()

,0
design,/content
design_name,principal
config,RUN_2023.04.03_04.09.15
flow_status,flow completed
total_runtime,0h3m10s0ms
routed_runtime,0h2m19s0ms
(Cell/mm^2)/Core_Util,4111.111111
DIEAREA_mm^2,0.09
CellPer_mm^2,2055.555556
OpenDP_Util,2.42


# Simulation

In [ ]:
!apt-get install iverilog

Reading package lists... Done
Building dependency tree       
Reading state information... Done
iverilog is already the newest version (10.3-1build1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [ ]:
import random

data_amount = 100

binary_keys = ["0000000","1000000","0100000","0010000", "0001000", "0000100","0000010","0000001" ]
frequencies = ["0", "1975", "1760", "1567", "1396", "1318", "1174","1046"]    

f_data = open ('datos_prueba.txt','w')
f_frequencies = open ('frequencies_test.txt','w')

#musical_notes = ["Silence", "Si","La", "Sol", "Fa", "Mi", "Re", "Do"]

n = data_amount

keys_binary_test = [0]  * n
frequencies_test = [0]  * n

for i in range(n):
  number = random.randint(0, 7)
  keys_binary_test[i] = binary_keys[number]
  frequencies_test[i] = frequencies[number]        
  f_data.write(keys_binary_test[i] + "\n") 
  f_frequencies.write(frequencies_test[i] + "\n")
  frequencies_test[i] = int(frequencies_test[i])
      

#print("Ingrese cuantos numeros aleatorios desea obtener")
#n=int(input())



print(keys_binary_test)

f_frequencies.close()
f_data.close()

['0010000', '0000000', '0000100', '0100000', '0000010', '0001000', '1000000', '0000100', '1000000', '0100000', '0000001', '0000010', '0010000', '0000000', '0100000', '0100000', '0000100', '0010000', '0000010', '0010000', '0000100', '0000100', '0000100', '0000001', '0000010', '0100000', '0001000', '0000010', '1000000', '0000000', '1000000', '0001000', '0000100', '0100000', '1000000', '0100000', '0100000', '0010000', '0100000', '0000000', '0100000', '0000000', '0010000', '0000000', '0010000', '0000100', '0100000', '0001000', '0000010', '0100000', '0001000', '0100000', '1000000', '0000000', '0010000', '0000001', '0000100', '0000010', '0000100', '0100000', '0100000', '0000001', '0010000', '0000010', '0000000', '0000001', '0000100', '1000000', '1000000', '0000000', '0010000', '0000001', '0000010', '0010000', '0000100', '0000100', '0000000', '0000000', '0010000', '1000000', '0000010', '0000100', '0000100', '0000100', '0000001', '0000010', '0001000', '0000010', '0000100', '0000100', '0000001'

In [ ]:
f_times = open ('tiempos_prueba.txt','w')

musical_notes = ["Silence", "Si","La", "Sol", "Fa", "Mi", "Re", "Do"]

n = data_amount

test_times = [0]  * n

for i in range(n):
      number = random.randint(2000000,10000000)
      number_bin = bin(number)[2:]
      test_times[i] = number_bin
      f_times.write(test_times[i] + "\n")

#print("Ingrese cuantos numeros aleatorios desea obtener")
#n=int(input())

print(test_times)

f_times.close()

['11101111010001001000100', '1011100000100101010110', '10100000101100110000110', '10010111101111011100110', '100101010010001010110011', '10001000000000010000101', '100101111010101001010111', '100100001000011101101100', '1000101000110010100001', '11011100011101010101111', '11011001011100101011111', '1100001000011100111011', '1011100100110011111000', '11001011110100101110100', '10100011101010111001010', '100000000010110000110110', '100101100001001001100010', '1001000001001110100110', '100100110001101001110111', '1101101011011001110111', '1100010000110000011110', '10100011001010111001110', '10000101001100101111010', '11111001011100110111001', '11100000011101110001110', '11100100001111010110001', '1000111001111101001001', '10111000111110011101001', '1000011010101110111100', '11100101101000000001011', '11101100000100000100011', '1110000001100100111010', '11111110100110000011001', '11011111101001110110110', '1011000110101001000111', '1011011000101111010010', '1001010100100001011011', '111101

In [ ]:
!iverilog -o sim_principal musica/test/sim_principal.v musica/rtl/principal.v  

In [ ]:
!vvp sim_principal

VCD info: dumpfile sim_principal.vcd opened for output.
              638200
          0
              758760
              568200
              851800
              716360
              506360
              758760
              506360
              568200
              956040
              851800
              638200
          0
              568200
              568200
              758760
              638200
              851800
              638200
              758760
              758760
              758760
              956040
              851800
              568200
              716360
              851800
              506360
          0
              506360
              716360
              758760
              568200
              506360
              568200
              568200
              638200
              568200
          0
              568200
          0
              638200
          0
              638200
              758760
              568200
           

# Comparación resultados simulacion

In [ ]:
file_out = open('output.txt','r')

measured_frequencies = [0] * data_amount

lines = file_out.readlines()
i = 0
for line in lines:
   number = int(line) * pow(10,-12)
   if number != 0 :
    measured_frequencies[i] = 1/number
    print(measured_frequencies[i])
   else: 
        measured_frequencies[i] = number
        print(measured_frequencies[i])
   i+=1
file_out.close()


1566.9069257286117
0.0
1317.9397965100954
1759.9436818021823
1173.984503404555
1395.9461723155955
1974.879532348527
1317.9397965100954
1974.879532348527
1759.9436818021823
1045.9813396928998
1173.984503404555
1566.9069257286117
0.0
1759.9436818021823
1759.9436818021823
1317.9397965100954
1566.9069257286117
1173.984503404555
1566.9069257286117
1317.9397965100954
1317.9397965100954
1317.9397965100954
1045.9813396928998
1173.984503404555
1759.9436818021823
1395.9461723155955
1173.984503404555
1974.879532348527
0.0
1974.879532348527
1395.9461723155955
1317.9397965100954
1759.9436818021823
1974.879532348527
1759.9436818021823
1759.9436818021823
1566.9069257286117
1759.9436818021823
0.0
1759.9436818021823
0.0
1566.9069257286117
0.0
1566.9069257286117
1317.9397965100954
1759.9436818021823
1395.9461723155955
1173.984503404555
1759.9436818021823
1395.9461723155955
1759.9436818021823
1974.879532348527
0.0
1566.9069257286117
1045.9813396928998
1317.9397965100954
1173.984503404555
1317.93979651009

# Total Error in the musical box

In [ ]:
import math
errors_abs = [0] * data_amount
sum = 0
for i in range(data_amount):
  if(frequencies_test[i] != 0):
    errors_abs[i] = abs(measured_frequencies[i] - frequencies_test[i])/frequencies_test[i]
    #print(errors_abs[i])
    sum += pow(frequencies_test[i] - measured_frequencies[i] , 2)
    
    sum = sum/data_amount

print(sum)


3.626896731514551e-05


# Improvements for next sessions

The initial project was built in VHDL and had more functions as a disk, volume buttons, and the option to record the melodies to listen to them later, so one way to complement the project implemented in this notebook would be to implement the other modules in Verilog to be included in this work.
A ring oscillator could be added to generate a clock inside the chip and not have to add a PAD.

